This notebook should be run in the solarjethunterenv.    

Its goal is to extract the wavelength information to see how much of the SOL events contained the 304 A chanel in the HEK reports - to answer the comments from the first referee report to the jet paper.

## Read SJH catalogue and extract SOL ids

In [307]:
from utils.Jet_class import json_export_list, json_import_list, csv_import_list, csv_export_list
import numpy as np 
from astropy.io import ascii
import matplotlib.pyplot as plt

In [308]:
json_file = 'exports/Jet_clusters_3.0_2.0_paperID_cluster_xy.json'

In [309]:
Jet_clusters=json_import_list(json_file)

The 881 JetCluster objects are imported from exports/Jet_clusters_3.0_2.0_paperID_cluster_xy.json.


In [310]:
Cluster_SOL = np.array([Jet_clusters[i].SOL for i in range(len(Jet_clusters))], dtype=str)
Cluster_date = np.array([Jet_clusters[i].obs_time for i in range(len(Jet_clusters))], dtype='datetime64')
Cluster_dur = np.array([Jet_clusters[i].Duration for i in range(len(Jet_clusters))], dtype=float)
sjh_identifier = np.array([Jet_clusters[i].ID for i in range(len(Jet_clusters))], dtype=str)

### Create the list of unique SOL ids

In [497]:
unique_SOL_events = np.unique(Cluster_SOL)
len(unique_SOL_events)

229

## Find the corresponding HEK events

In [312]:
print(np.min(Cluster_date))
print(np.max(Cluster_date))

2011-01-20T09:15:44.000000
2016-12-24T14:29:42.000000


In [313]:
from sunpy.net import attrs as a, Fido
timerange = a.Time('2011/01/01 00:00:00', '2017/01/01 00:00:00')
res = Fido.search(timerange, a.hek.CJ) 

In [510]:
hek_SOL = res['hek']["SOL_standard"]
unique_hek_SOL = np.unique(hek_SOL)
len(unique_hek_SOL)

367

In [514]:
selection_tot304 = res['hek'][res['hek']["obs_meanwavel"]==3.04e-06]
unique_hek_SOL_304 = np.unique(selection_tot304["SOL_standard"])
len(unique_hek_SOL_304)

123

### Example with the first SOL id in our list

In [10]:
selection0 = res['hek'][res['hek']["SOL_standard"]==unique_SOL_events[0]]

In [21]:
print(selection0["obs_meanwavel"].value)

[2.11e-06 3.04e-06]


In [16]:
selection0

gs_thumburl,comment_count,hpc_bbox,outflow_width,frm_humanflag,hgc_coord,event_coordsys,obs_levelnum,hpc_coord,event_npixels,gs_imageurl,ar_polarity,frm_paramset,hrc_coord,event_starttime,ar_mtwilsoncls,event_type,intensmin,obs_meanwavel,outflow_openingangle,frm_url,skel_chaincode,bound_chaincode,noposition,active,intensmax,frm_versionnumber,area_uncert,obs_dataprepurl,hpc_geom,hgc_bbox,intensmedian,chaincodetype,obs_channelid,event_clippedspatial,ar_noaaclass,SOL_standard,event_avg_rating,eventtype,intensunit,hpc_boundcc,event_mapurl,frm_contact,ar_penumbracls,intensmean,bound_ccstartc1,outflow_transspeed,frm_name,area_atdiskcenter,frm_identifier,obs_observatory,event_description,boundbox_c2ur,obs_firstprocessingdate,boundbox_c2ll,frm_institute,hrc_bbox,refs_orig,ar_mcintoshcls,event_maskurl,bound_ccstartc2,gs_movieurl,event_score,skel_startc2,skel_startc1,event_expires,outflow_speedunit,hrc_boundcc,event_probability,intensvar,frm_daterun,outflow_lengthunit,outflow_length,event_coordunit,hpc_y,hpc_x,search_instrument,ar_numspots,kb_archivdate,kb_archivist,intenstotal,sum_overlap_scores,hgs_boundcc,intensskew,obs_includesnrt,rasterscan,obs_wavelunit,kb_archivid,search_frm_name,boundbox_c1ur,ar_noaanum,area_atdiskcenteruncert,boundbox_c1ll,event_importance_num_ratings,ar_compactnesscls,skel_curvature,event_testflag,event_c2error,hrc_r,skel_nsteps,hgs_y,obs_title,hgs_x,hcr_checked,frm_specificid,event_title,obs_instrument,event_c1error,revision,hpc_radius,outflow_widthunit,event_endtime,event_importance,event_coord2,event_coord3,event_coord1,search_observatory,area_raw,concept,event_pixelunit,hgc_boundcc,outflow_speed,hgc_x,hrc_a,event_peaktime,hgc_y,gs_galleryid,hgs_coord,ar_zurichcls,bound_ccnsteps,intenskurt,event_clippedtemporal,rasterscantype,search_channelid,hgs_bbox,area_unit,obs_lastprocessingdate,refs
str194,str1,str130,object,str5,str35,str12,object,str30,object,str188,object,str14,str41,Time,str1,str2,object,float64,object,str61,str1,str1,str5,str4,object,object,object,str1,str186,str125,object,str1,str8,str1,str1,str30,object,str2,str1,str1,str1,str12,str1,object,object,object,str22,object,str29,str4,str560,float64,str1,float64,str23,str110,str1,str1,str1,object,str212,str1,object,object,str1,str6,str1,object,object,str19,str2,object,str6,float64,float64,str7,object,str19,str19,object,str22,str1,object,str1,str1,str2,str82,str16,float64,object,object,float64,str1,str1,object,str5,float64,float64,object,float64,str1,float64,str5,str1,str68,str7,float64,str1,str19,str2,Time,object,float64,object,float64,str4,object,str11,str1,str1,object,float64,float64,str19,float64,str48,str32,str1,object,object,str1,str1,str16,str120,str1,str1,object
http://www.lmsal.com/hpkb/podimages/2011/01/21/pod_sainz%2Bdalda_alberto_2011-01-21T22%3A42%3A32.088/thumb/panorama_asainz_AIA-304_AIA-211_20110120T094545_at_20110121T224035.jpg,0,"POLYGON((-335 -1119,-155 -1119,-155 -887,-335 -887,-335 -1119))",None,true,POINT(-71.12168289 -75.492392),UTC-HPC-TOPO,None,POINT(-245 -1003),None,http://www.lmsal.com/hpkb/podimages/2011/01/21/pod_sainz%2Bdalda_alberto_2011-01-21T22%3A42%3A32.088/panorama_asainz_AIA-304_AIA-211_20110120T094545_at_20110121T224035.jpg,None,n/a,POINT(1.05939829132651 166.273314931505),2011-01-20 09:00:09.000,,CJ,None,2.11e-06,None,n/a,,,false,true,None,None,None,,010300000001000000050000000000000000F074C000000000007C91C000000000006063C000000000007C91C000000000006063C00000000000B88BC00000000000F074C00000000000B88BC00000000000F074C000000000007C91C0,"POLYGON((-74.395154 -72.691952,-58.959908 -80.8028,-27.277949 -69.861778,-65.488232 -67.863333,-74.395154 -72.691952))",None,,211,,,SOL2011-01-20T09:00:09L289C165,None,14,,,,Scott Green,,None,None,None,asainz,None,Annotator-build_20100909-avc,SDO,Coronal jet in the South pole. It seems like a twisted loop. High temporal resolution.,-1119.0,,-887.0,LMSAL,"POLYGON((1.198512 163.333644,1.159126 172.113769,0.923909 170.087857,0.972864 159.309574,1.198512 163.333644))",,,,None,http://sdoww

In [22]:
3.04e-06 in selection0["obs_meanwavel"].value

True

### Loop

In [314]:
reported_in_304 = []
for sol_id in unique_SOL_events:
    selection = res['hek'][res['hek']["SOL_standard"]==sol_id]
    reported_in_304.append(3.04e-06 in selection["obs_meanwavel"].value)
reported_in_304 = np.array(reported_in_304)

In [315]:
len(reported_in_304)

229

In [316]:
onlyTrue = unique_SOL_events[reported_in_304]

In [317]:
np.shape(onlyTrue)

(102,)

In [318]:
type(onlyTrue)

numpy.ndarray

Make sure we selected SOL events associated with reporst in 304 A

In [319]:
reported_in_304_check = []
for sol_id in onlyTrue:
    selection = res['hek'][res['hek']["SOL_standard"]==sol_id]
    reported_in_304_check.append(3.04e-06 in selection["obs_meanwavel"].value)

In [320]:
len(reported_in_304_check)

102

# Select SJH jets from these 304 A HEK events

In [ ]:
# DO NOT USE I think there is a mistake somewhere

In [322]:
#Jet_clusters_304 = []
#for cluster in Jet_clusters:
#    if cluster.SOL in onlyTrue:
#        Jet_clusters_304.append(cluster)
#Jet_clusters_304 = np.array(Jet_clusters_304)

In [323]:
#np.shape(Jet_clusters_304)

(448,)

In [324]:
#Cluster_SOL_304 = np.array([Jet_clusters[i].SOL for i in range(len(Jet_clusters_304))], dtype=str)
#Cluster_date_304 = np.array([Jet_clusters[i].obs_time for i in range(len(Jet_clusters_304))], dtype='datetime64')
#Cluster_dur_304 = np.array([Jet_clusters[i].Duration for i in range(len(Jet_clusters_304))], dtype=float)

In [391]:
#len(Cluster_SOL_304)

### Find out the proportion of time when there is actually a jet in the HEK reports

Extract the time intervals for each jet that belong to a selected HEK SOL event, and sort them by starting time.

In [459]:
SOL = 'SOL2015-05-29T00:50:23L331C065'
SOL = 'SOL2015-07-17T08:00:06L032C049'
#SOL = onlyTrue[1]
jet_start = Cluster_date[Cluster_SOL == SOL]
jet_dur = Cluster_dur[Cluster_SOL == SOL]
jet_start = Cluster_date[Cluster_SOL == SOL]
jet_start

array(['2015-07-17T08:20:54.000000'], dtype='datetime64[us]')

In [460]:
jet_end = []
for i in range(len(jet_start)):
    jet_end.append( jet_start[i] + np.timedelta64(int(jet_dur[i]*60),'s') )
jet_end = np.array(jet_end)

In [461]:
jet_times = np.dstack((jet_start, jet_end))
jet_times_sorted = np.sort(jet_times, axis=1)
jet_times_sorted[0,-1,1]

numpy.datetime64('2015-07-17T08:37:06.000000')

In [462]:
jet_times_sorted

array([[['2015-07-17T08:20:54.000000', '2015-07-17T08:37:06.000000']]],
      dtype='datetime64[us]')

In [463]:
no_overlap = True
for i in range(len(jet_start)-1):
    if jet_times_sorted[0,i,1]>jet_times_sorted[0,i+1,0]:
        no_overlap = False
no_overlap

True

Because the jets do not overlap then we can sum the duration of all clusters 

In [464]:
print(jet_dur)
print(jet_dur.sum())

[16.2]
16.2


Now extract time from the HEK CJ report

In [465]:
selection1 = res['hek'][res['hek']["SOL_standard"]==SOL]
start_time = selection1['event_starttime'][0]
end_time = selection1['event_endtime'][0]

Check that the clusters start and end both in the time interval selected

In [466]:
print(jet_start.min() > start_time)
print(jet_end.max() < end_time)

True
True


Now calculate duration of interval

In [467]:
timedelta = end_time - start_time
print(timedelta.datetime.total_seconds()/60.)
print(jet_dur.sum() / (timedelta.datetime.total_seconds()/60.))

60.0
0.26999999999999996


In [468]:
def calculate_jet_total_duration(jet_start, jet_end, jet_dur):
    # sort the jet times
    jet_times = np.dstack((jet_start, jet_end))
    jet_times_sorted = np.sort(jet_times, axis=1)
    
    # check that they do not overlap
    no_overlap = True
    if len(jet_start) > 1:
        for i in range(len(jet_start)-1):
            if jet_times_sorted[0,i,1]>jet_times_sorted[0,i+1,0]:
                no_overlap = False
                #print("there is an overlap!!!")
        
    if no_overlap:
        jet_time = jet_dur.sum() # in minutes
    else:
        jet_time = jet_dur[0] # initialisation of jet 1
        for i in range(len(jet_start)-1):
            if jet_times_sorted[0,i,1]>jet_times_sorted[0,i+1,0]: # in this case there is overlap
                additional_time = jet_times_sorted[0,i+1,1] - jet_times_sorted[0,i,1]
                jet_time = jet_time + additional_time.astype('timedelta64[m]').astype('float')
            else:
                jet_time = jet_time + jet_dur[i+1]
    
    return jet_time

In [469]:
jet_dur

array([16.2])

In [470]:
calculate_jet_total_duration(jet_start, jet_end, jet_dur)

16.2

In [487]:
def calculate_nojet_time(start_time, end_time, jet_start, jet_end, jet_dur):
    error_in_timing = False
    # check that the intervals make sense:
    if jet_start.min() < start_time:
        print("There might be an error in the timing")
        print("Jet min start:")
        print(jet_start.min())
        print("and HEK report start:")
        print(start_time)
        error_in_timing = True
    if jet_end.max() > end_time:
        print("There might be an error in the timing")
        print("Jet max end:")
        print(jet_end.max())
        print("and HEK report end:")
        print(end_time)
        error_in_timing = True

    jet_time = calculate_jet_total_duration(jet_start, jet_end, jet_dur) # in minutes
    
    #calculate HEK event time
    timedelta = (end_time - start_time).datetime.total_seconds()/60. # in minutes

    # calculate no jet time
    no_jet_time = (timedelta - jet_time)/timedelta
    
    return no_jet_time, error_in_timing, jet_time, timedelta

In [488]:
jet_time = jet_dur.sum()
type(jet_time)
timedelta = (end_time - start_time).datetime.total_seconds()/60.
type(timedelta)
no_jet_time = (timedelta - jet_time)/timedelta
no_jet_time

0.7599999999999999

In [489]:
calculate_nojet_time(start_time, end_time, jet_start, jet_end, jet_dur)

(0.7599999999999999, False, 26.4, 110.0)

In [491]:
error_in_timing = []
no_jet_time = [] # ratio
jet_time = [] # in minutes
hek_time = [] # in minutes
for SOL in onlyTrue:
    print(SOL)
    jet_start = Cluster_date[Cluster_SOL == SOL]
    jet_dur = Cluster_dur[Cluster_SOL == SOL]
    jet_end = []
    for i in range(len(jet_start)):
        jet_end.append( jet_start[i] + np.timedelta64(int(jet_dur[i]*60),'s') )
    jet_end = np.array(jet_end)
    selection1 = res['hek'][res['hek']["SOL_standard"]==SOL]
    start_time = selection1['event_starttime'][0]
    end_time = selection1['event_endtime'][0]
    nojt, err, jt, ht = calculate_nojet_time(start_time, end_time, jet_start, jet_end, jet_dur)
    print(nojt)
    error_in_timing.append(err)
    no_jet_time.append(nojt)
    jet_time.append(jt)
    hek_time.append(ht)

SOL2011-01-20T09:00:09L289C165
0.26599326599326595
SOL2011-01-20T22:00:09L353C074
0.971111111111111
SOL2011-01-24T22:15:09L358C011
There might be an error in the timing
Jet min start:
2011-01-24T22:15:08.000000
and HEK report start:
2011-01-24 22:15:09.000
0.3009523809523809
SOL2011-01-26T01:52:21L007C038
0.48856209150326796
SOL2011-02-13T04:57:46L131C056
0.3775510204081632
SOL2011-02-14T05:31:16L307C065
0.22549019607843143
SOL2011-03-20T00:00:04L223C126
0.6600790513833992
SOL2011-04-01T03:00:11L222C115
0.735
SOL2011-05-27T07:40:18L038C079
0.39
SOL2011-05-27T21:40:29L028C078
0.3333333333333333
SOL2011-05-28T00:00:02L027C078
0.1433333333333332
SOL2011-05-29T07:30:02L165C104
0.598
SOL2011-05-29T16:10:18L064C064
0.7333333333333333
SOL2011-08-01T00:20:04L291C071
0.8565853658536585
SOL2011-09-14T10:30:04L049C062
0.8891891891891891
SOL2011-12-11T02:00:04L352C108
0.6766666666666666
SOL2011-12-11T11:30:00L353C109
0.6163934426229508
SOL2011-12-11T22:30:00L351C107
0.47704918032786886
SOL2012-01-

In [476]:
len(error_in_timing)

102

In [493]:
np.array(hek_time).sum()

20040.6

In [494]:
np.array(jet_time).sum()

8890.6

In [496]:
np.array(jet_time).sum() / np.array(hek_time).sum() * 100

44.36294322525274

In [364]:
jet_times_sorted[0,i,1]

numpy.datetime64('2011-01-21T00:00:08.000000')

In [456]:
onlyTrue[66]

'SOL2015-05-08T08:02:03L084C070'

In [457]:
onlyTrue[67]

'SOL2015-05-29T00:50:23L331C065'

In [458]:
onlyTrue[68]

'SOL2015-07-17T08:00:06L032C049'